# DREAM data reduction

In [ ]:
from pathlib import Path

import scipp as sc
import scippneutron as scn
import sciline
from ess import dream, powder
from ess.powder.types import *
from ess.dream.io.geant4 import providers as geant4_providers

In [ ]:
providers = (
    *geant4_providers,
    *powder.providers,
    *dream.providers,
)
params = {
    FilePath[SampleRun]: dream.data.simulated_diamond_sample(),
    FilePath[VanadiumRun]: dream.data.simulated_vanadium_sample(),
    FilePath[EmptyCanRun]: dream.data.simulated_empty_can(),
    DetectorName: DetectorName('mantle'),

    # The upper bounds mode is not yet implemented.
    UncertaintyBroadcastMode: UncertaintyBroadcastMode.drop,

    # Crop data to this range in time-of-flight
    ValidTofRange: sc.array(dims=['tof'], values=[0.0, 86_000.0], unit='us'),

    # Edges for binning in d-spacing
    DspacingBins: sc.linspace('dspacing', 0.0, 2.3434, 200, unit='angstrom'),

    # Not available in simulated data
    RawSample[SampleRun]: sc.DataGroup(position=sc.vector([0., 0., 0.], unit='mm')),
    RawSample[VanadiumRun]: sc.DataGroup(position=sc.vector([0., 0., 0.], unit='mm')),
    RawSource: sc.DataGroup(position=sc.vector([-3.478, 0.0, -76550], unit='mm')),
    AccumulatedProtonCharge[SampleRun]: sc.scalar(1.0, unit='µAh'),
    AccumulatedProtonCharge[VanadiumRun]: sc.scalar(1.0, unit='µAh'),
}

In [ ]:
pipeline = sciline.Pipeline(providers, params=params)

We don't have calibration data yet.
So convert to d-spacing without calibration.

In [ ]:
from ess.powder.conversion import to_dspacing_with_positions
pipeline.insert(to_dspacing_with_positions)

In [ ]:
pipeline.visualize(DspacingHistogram, graph_attr={'rankdir': 'LR'})

In [ ]:
result = pipeline.compute(DspacingHistogram)

In [ ]:
result

In [ ]:
result.plot()

In [ ]:
result.coords['dspacing'] = sc.midpoints(result.coords['dspacing'])
scn.io.save_xye('dspacing.xye', result)